## Problem 1

In [4]:
import torch
import numpy as np
from torch.autograd import Variable
import matplotlib.pyplot as plt
from IPython import display

X1 = np.array([[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]])
X2 = np.flip(X1, axis=1).copy()
a = np.array(([[8.07131, 1730.63, 233.426], [7.43155, 1554.679, 240.337]]))
T = 20
pwat = 10**(a[0, 0]- a[0,1] / (T + a[0,2]))
pdio = 10**(a[1, 0]- a[1,1] / (T + a[1,2]))
P = np.array([[28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5]])
P = torch.tensor(P, requires_grad=False, dtype=torch.float32)

A = Variable(torch.tensor([1.0, 1.0]), requires_grad=True) #includes initial guess

x1 = torch.tensor(X1, requires_grad=False, dtype=torch.float32)
x2 = torch.tensor(X2, requires_grad=False, dtype=torch.float32)

#simple gradient descent
a = 0.0001

#fixed number of iterations, need to add termination criteria
for i in range(100):
    P_pred = x1 * torch.exp(A[0] * (A[1] * x2 / (A[0] * x1 + A[1] * x2 )) ** 2) * pwat + \
        x2 * torch.exp(A[1] * (A[0] * x1 / (A[0] * x1 + A[1] * x2 )) ** 2) * pdio
    
    loss = (P_pred - P) ** 2
    loss = loss.sum() 
    
    loss.backward()
    
    with torch.no_grad():
        A -= a * A.grad #A = A -  a * A.grad
        
        A.grad.zero_() #to not keep current gradient
        
        
print('estimation A12 and A21 is:' , A)
print('final loss is:', loss.data.numpy())



estimation A12 and A21 is: tensor([1.9110, 1.7293], requires_grad=True)
final loss is: 0.87836564


## Problem 2